In [1]:
import numpy as np
import random
import os
import math
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler


from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from torch.autograd import Variable

import torchvision.models as models
from torchvision import transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.empty_cache()

### data

In [5]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    image_path = os.path.join(data_dir, 'image')
    label_path = os.path.join(data_dir, 'label')
    
    for product_name in os.listdir(image_path):
        product_path = os.path.join(image_path, product_name)
        if os.path.isdir(product_path):
            # get image path
            img_path_list.extend(glob(os.path.join(product_path, '*.jpg')))
            img_path_list.extend(glob(os.path.join(product_path, '*.png')))
            label = list(product_name[:5])
            
            # get label
            label_list.append(''.join(label))
                
    return img_path_list, label_list

In [6]:
img_list, label_list = get_train_data('./Data/product_image/Training/')

In [7]:
def data_blanced(img, label):
    x = []
    y = []
    
    for i in range(len(label)):
        _img = img[(i * 114): ((i + 1) * 114)]
        _label = label[i]
        
        for img_product in _img:
            x.append(img_product)
            y.append(_label)
            
    return x, y

In [8]:
x, y = data_blanced(img_list, label_list)

In [9]:
le = preprocessing.LabelEncoder()
targets = le.fit_transform(y)
targets = torch.as_tensor(targets)

In [10]:
len(targets)

8664

In [11]:
one_hot_y = F.one_hot(targets)

In [12]:
one_hot_y.shape

torch.Size([8664, 76])

In [13]:
def get_valid_data(data_dir):
    img_valid_list = []
    label_valid_list = []
    
    image_path = os.path.join(data_dir, 'image')
    label_path = os.path.join(data_dir, 'label')
    
    for product_name in os.listdir(image_path):
        product_path = os.path.join(image_path, product_name)
        if os.path.isdir(product_path):
            # get image path
            img_valid_list.extend(glob(os.path.join(product_path, '*.jpg')))
            img_valid_list.extend(glob(os.path.join(product_path, '*.png')))
            label = list(product_name[:5])
            
            # get label
            label_valid_list.append(''.join(label))
                
    return img_valid_list, label_valid_list

In [14]:
def valid_data_blanced(img, label):
    x = []
    y = []
    
    for i in range(len(label)):
        _img = img[(i * 15): ((i + 1) * 15)]
        _label = label[i]
        
        for img_product in _img:
            x.append(img_product)
            y.append(_label)
            
    return x, y

In [15]:
img_valid_list, label_valid_list = get_valid_data('./Data/product_image/Validation/')

In [16]:
x_valid, y_valid = valid_data_blanced(img_valid_list, label_valid_list)

In [17]:
len(label_valid_list)

76

In [18]:
le2 = preprocessing.LabelEncoder()
targets_y = le2.fit_transform(y_valid)
targets_y = torch.as_tensor(targets_y)
one_hot_valid_y = F.one_hot(targets_y)

In [19]:
one_hot_valid_y.shape

torch.Size([1140, 76])

In [21]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image)

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [22]:
train_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((256, 256)),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])

test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((256, 256)),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])

In [23]:
train_dataset = CustomDataset(x, one_hot_y, train_mode=True, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True, num_workers=0, collate_fn=None)

vali_dataset = CustomDataset(x_valid, one_hot_valid_y, train_mode=True, transforms=test_transform)
vali_loader = DataLoader(vali_dataset, batch_size = 5, shuffle=False, num_workers=0, collate_fn=None)

### model

In [24]:
class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        model = models.resnet50(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(2048, 1000)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(1000,76)

    def forward(self, x):
        x = self.feature_extract(x)
        # x = x.mean(dim=(-2, -1))
        # (batch, 2048, 4, 4)
        x = torch.squeeze(x)
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
        return out

### training

In [25]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    # regressor = nn.Linear(1000, 1).to(device)

    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            # logit = regressor(logit)
            logit = torch.squeeze(logit)
            loss = criterion(logit, label)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

In [26]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    
    model.to(device)
    
    epochs = 30

    loss_plot = []
    vali_loss_plot = []

    # Loss Function
    criterion = torch.nn.CrossEntropyLoss()
    best_loss = 9999
    
    for epoch in range(1,epochs+1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            logit = torch.squeeze(logit)
            # Calc loss
            loss = criterion(logit, label)
            loss_plot.append(loss)

            # backpropagation
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            
        if scheduler is not None:
            scheduler.step()
            
        # Evaluation Validation set
        vali_loss = validation(model, vali_loader, criterion, device)
        vali_loss_plot.append(vali_loss)
        
        print(f'Epoch [{epoch}] Train loss : [{np.mean(train_loss):.5f}] Validation loss : [{vali_loss:.5f}]\n')
        
        # Model Saved
        if best_loss > vali_loss:
            best_loss = vali_loss
            torch.save(model.state_dict(), './saved_models/resnet50_ver3.pth')
            print('Model Saved.')

In [26]:
model = ResNet50()

# model_state_dict = torch.load("./saved_models/resnet50.pth", map_location=device)
# model.load_state_dict(model_state_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
scheduler = None

train(model, optimizer, train_loader, vali_loader, scheduler, device)

  0%|          | 0/271 [00:00<?, ?it/s]

  0%|          | 0/228 [00:00<?, ?it/s]

Epoch [1] Train loss : [3.77912] Validation loss : [3.80117]

Model Saved.


  0%|          | 0/271 [00:00<?, ?it/s]

  0%|          | 0/228 [00:00<?, ?it/s]

...

  0%|          | 0/228 [00:00<?, ?it/s]

Epoch [29] Train loss : [0.09402] Validation loss : [0.18097]



  0%|          | 0/271 [00:00<?, ?it/s]

  0%|          | 0/228 [00:00<?, ?it/s]

Epoch [30] Train loss : [0.09177] Validation loss : [0.07926]

Model Saved.


In [29]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img, label in tqdm(iter(test_loader)):
            img = img.float().to(device)
            
            pred_logit = model(img)
            pred_logit = pred_logit.squeeze().detach().cpu()
            
            model_pred.extend(pred_logit.tolist())
    return model_pred

In [31]:
checkpoint = torch.load('./saved_models/resnet50_ver3.pth')
model = ResNet50().to(device)
model.load_state_dict(checkpoint)

preds = predict(model, vali_loader, device)

  0%|          | 0/228 [00:00<?, ?it/s]

In [78]:
pred_labels = np.argmax(preds, axis=1)

In [79]:
true_labels = one_hot_valid_y.argmax(-1)

In [80]:
from sklearn.metrics import accuracy_score

accuracy_score(true_labels, pred_labels)

0.9596491228070175